In [1]:
# Requirements if running from Google Colab with a T4 GPU. 
!pip install transformers torch lm-format-enforcer huggingface_hub optimum
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.1 MB/s eta 0:00:00
  Using cached datasets-2.16.1-py3-none-any.whl.metadata (20 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.9 MB/s eta 0:00:00
  Using cached pyarrow-15.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2023.10.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached aiohttp-3.9.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Usi

In [17]:
from pydantic import BaseModel
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn
from transformers import pipeline

class MusicGenInfoFormat(BaseModel):
    tone: str
    intensity: str
    instrumentation: str
    setting: str
    scene_description: str
    pitch: str
    crescendo: bool

text = """The dark figure streaming with fire raced towards them. The orcs yelled and poured over the stone gangways. Then Boromir raised his horn and blew. Loud the challenge rang and bellowed, like the shout of many throats under the cavernous roof. For a moment the orcs quailed and the fiery shadow halted. Then the echoes died as suddenly as a flame blown out by a dark wind, and the enemy advanced again.

'Over the bridge!' cried Gandalf, recalling his strength. 'Fly! This is a foe beyond any of you. I must hold the narrow way. Fly!' Aragorn and Boromir did not heed the command, but still held their ground, side by side, behind Gandalf at the far end of the bridge. The others halted just within the doorway at the hall's end, and turned, unable to leave their leader to face the enemy alone.
"""

prompt = f"""
{text}\n\n In the following format {MusicGenInfoFormat.schema_json()}, A piece music generated as background ambience for the above text would have these qualities: \n
"""

# Create a transformers pipeline
hf_pipeline = pipeline('text-generation', model='TheBloke/Llama-2-7b-Chat-GPTQ', device_map='auto')

# Create a character level parser and build a transformers prefix function from it
parser = JsonSchemaParser(MusicGenInfoFormat.schema())
prefix_function = build_transformers_prefix_allowed_tokens_fn(hf_pipeline.tokenizer, parser)

# Call the pipeline with the prefix function
output_dict = hf_pipeline(prompt, prefix_allowed_tokens_fn=prefix_function)

# Extract the results
result = output_dict[0]['generated_text'][len(prompt):]
print(result)
# {'first_name': 'Michael', 'last_name': 'Jordan', 'year_of_birth': 1963, 'num_seasons_in_nba': 15}













{
"tone": "Dark",
"intensity": "Loud",
"instrumentation": "Ethereal",
"setting": "Dark, ominous",
"scene_description": "A dark figure races towards the bridge",
"pitch": "C",
"crescendo": true
}














In [18]:
class LLMPromptGenerator:
    
    class MusicGenInfo(BaseModel):
        tone: str
        intensity: str
        instrumentation: str
        setting: str
        scene_description: str
        pitch: str
        crescendo: bool

    def __init__(self, model_name="TheBloke/Llama-2-7b-Chat-GPTQ"):
        self.model_name = model_name
        self.hf_pipeline = pipeline('text-generation', model=model_name, device_map='auto')

    def generate_prompt(self, text, music_gen_info=MusicGenInfo):
        prompt = f"""
        {text}\n\n In the following format {music_gen_info.schema_json()}, A piece music generated as background ambience for the above text would have these qualities: \n
        """
        return prompt
    
    def extract_info(self, text, music_gen_info=MusicGenInfo):
        prompt = self.generate_prompt(text, music_gen_info)
        parser = JsonSchemaParser(music_gen_info.schema())
        prefix_function = build_transformers_prefix_allowed_tokens_fn(self.hf_pipeline.tokenizer, parser)
        output_dict = self.hf_pipeline(prompt, prefix_allowed_tokens_fn=prefix_function)
        result = output_dict[0]['generated_text'][len(prompt):]
        return result
    

In [19]:
LLMPromptGenerator().extract_info(text)

'\n\n\n\n\n\n\n\n\n\n\n\n{\n"tone": "Dark",\n"intensity": "Loud",\n"instrumentation": "Ethereal",\n"setting": "Dark, ominous",\n"scene_description": "A dark, ominous landscape",\n"pitch": "C",\n"crescendo": false\n}\n\n\n\n\n\n\n\n\n\n\n\n'